In [1]:
import torch
import gc
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer, SFTConfig

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"
ADAPTER_NAME = "mistral-7b-kaira"

#  4-Bit Quantization Config 
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
CSV_FILE_PATH = "french_argot_train.csv" 

#MAX_LENGTH_LIMIT = 1024
MAX_LENGTH_LIMIT = 2048

In [2]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline


print(f"⏳ Loading Base Model: {MODEL_NAME}...")
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)

#  Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# 3. Attach Your LoRA Adapter
print(f"🔗 Attaching Adapter: {ADAPTER_NAME}...")
model = PeftModel.from_pretrained(base_model, ADAPTER_NAME)

print("✅ Kaira Model Ready!")

⏳ Loading Base Model: mistralai/Mistral-7B-Instruct-v0.3...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

🔗 Attaching Adapter: mistral-7b-kaira...
✅ Kaira Model Ready!


In [15]:
def translate_to_kaira(text, max_new_tokens=512):
    # Format the Prompt
    raw_prompt = f"Réécris le texte suivant en style Verlan: \n{text}"
    prompt = f"<s>[INST]  {raw_prompt} [/INST]"
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    # Generate
    # We use typical sampling parameters for creative writing
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,    # Creativity (0.1 = Robotic, 1.0 = Chaotic)
            top_p=0.9,          # Nucleus sampling
            repetition_penalty=1.1, # Prevent repeating words
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode
    # We strip the input prompt to show only the translation
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    #print(f"generated={generated_text}")

    if raw_prompt in generated_text:
        generated_text =  generated_text.split(raw_prompt)[-1].strip()

    if "\n" in generated_text:    
        generated_text =  generated_text.split("\n")[-1].strip()
    return generated_text

print("Function defined.")

Function defined.


In [17]:
# Test 1: Standard Greeting
text1 = "Bonjour, comment vas-tu mon ami ?"
print(f"🇫🇷 Standard: {text1}")
print(f"🧢 Kaira:    {translate_to_kaira(text1)}\n")

# Test 2: Conflict
text2 = "Il m'a énervé, je vais devoir me battre avec lui."
print(f"🇫🇷 Standard: {text2}")
print(f"🧢 Kaira:    {translate_to_kaira(text2)}\n")

# Test 3: Money/Situation
text3 = "Je n'ai plus d'argent, c'est une situation très difficile."
print(f"🇫🇷 Standard: {text3}")
print(f"🧢 Kaira:    {translate_to_kaira(text3)}\n")

🇫🇷 Standard: Bonjour, comment vas-tu mon ami ?
🧢 Kaira:    Wesh, comment t'es mon pote ?

🇫🇷 Standard: Il m'a énervé, je vais devoir me battre avec lui.
🧢 Kaira:    Il m'a vénéré, j'vais devoir me battre avec lui.

🇫🇷 Standard: Je n'ai plus d'argent, c'est une situation très difficile.
🧢 Kaira:    u Alors, alors j'ai plus de fric, c'est une vraie galère !  Mais faut pas se faire arnaquer, tu vois ?  Je suis vraiment dans la merde, t'sais.

